<a href="https://colab.research.google.com/github/Codehackerone/nlp-with-transformers/blob/main/Q%26A(SQuAD)(huggingface).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download SQuAD

In [18]:
url = 'https://raw.githubusercontent.com/kushalj001/pytorch-question-answering/master/data/'
files = ['squad_train.json', 'squad_dev.json']

In [3]:
import os

squad_dir = './data/squad'

In [6]:
%mkdir ./data
os.mkdir(squad_dir)

In [10]:
import requests

In [19]:
for file in files:
  res = requests.get(url+file)
  with open(os.path.join(squad_dir, file), 'wb') as fp:
    for chunk in res.iter_content(chunk_size=40):
      fp.write(chunk)

In [20]:
import json

with open(os.path.join(squad_dir, files[0]), 'rb') as f:
  squad = json.load(f)

In [21]:
squad['data'][0]['paragraphs'][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'id': '5733be284776f41900661182',
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

# Processing Training Data

In [22]:
new_squad = []

In [23]:
for group in squad['data']:
  for paragraph in group['paragraphs']:
    context = paragraph['context']
    for qa_pair in paragraph['qas']:
      question = qa_pair['question']
      if 'answers' in qa_pair.keys() and len(qa_pair['answers']) > 0:        
        answer = qa_pair['answers'][0]['text']
      elif 'plausible_answers' in qa_pair.keys() and len(qa_pair['plausible_answers']) > 0:
        answer = qa_pair['plausible_answers'][0]['text']
      else:
        answer = None

      new_squad.append({
          'question':question,
          'answer':answer,
          'context':context
      })

In [24]:
new_squad[:2]

[{'answer': 'Saint Bernadette Soubirous',
  'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
 {'answer': 'a copper statue of Christ',
  'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golde

In [25]:
with open(os.path.join(squad_dir, 'train.json'), 'w') as f:
  json.dump(new_squad, f)

In [26]:
import json

with open(os.path.join(squad_dir, files[1]), 'rb') as f:
  squad = json.load(f)

for group in squad['data']:
  for paragraph in group['paragraphs']:
    context = paragraph['context']
    for qa_pair in paragraph['qas']:
      question = qa_pair['question']
      if 'answers' in qa_pair.keys() and len(qa_pair['answers']) > 0:        
        answer = qa_pair['answers'][0]['text']
      elif 'plausible_answers' in qa_pair.keys() and len(qa_pair['plausible_answers']) > 0:
        answer = qa_pair['plausible_answers'][0]['text']
      else:
        answer = None

      new_squad.append({
          'question':question,
          'answer':answer,
          'context':context
      })

with open(os.path.join(squad_dir, 'val.json'), 'w') as f:
  json.dump(new_squad, f)

# QandA Model with HuggingFace


In [38]:
with open('data/squad/val.json', 'r') as f:
  squad = json.load(f)

In [31]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
     |████████████████████████████████| 596 kB 27.7 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [32]:
from transformers import BertTokenizer, BertForQuestionAnswering

In [33]:
modelname = 'deepset/bert-base-cased-squad2'

In [34]:
tokenizer = BertTokenizer.from_pretrained(modelname)
model = BertForQuestionAnswering.from_pretrained(modelname)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [35]:
from transformers import pipeline

In [36]:
qs = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [39]:
squad[0]

{'answer': 'Saint Bernadette Soubirous',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'}

In [41]:
qs({
    'question':'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
    'context':'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'
})

{'answer': 'Saint Bernadette Soubirous',
 'end': 541,
 'score': 0.9993393421173096,
 'start': 515}

In [43]:
answers = []

for pair in squad[:5]:
  ans = qs({
      'question':pair['question'],
      'context':pair['context']
  })

  answers.append({
      'predicted':ans['answer'],
      'true':pair['answer']
  })

In [44]:
answers

[{'predicted': 'Saint Bernadette Soubirous',
  'true': 'Saint Bernadette Soubirous'},
 {'predicted': 'a copper statue of Christ',
  'true': 'a copper statue of Christ'},
 {'predicted': 'the Main Building', 'true': 'the Main Building'},
 {'predicted': 'a Marian place of prayer and reflection.',
  'true': 'a Marian place of prayer and reflection'},
 {'predicted': 'a golden statue of the Virgin Mary.',
  'true': 'a golden statue of the Virgin Mary'}]

# Metrics

## Exact Match(EM)

In [45]:
em = []

for answer in answers:
  if answer['predicted'] == answer['true']:
    em.append(1)
  else:
    em.append(0)

In [46]:
# Accuracy
sum(em)/len(em)

0.6

In [47]:
import re
em = []

for answer in answers:
  pred = re.sub('[^0-9a-z ]', '', answer['predicted'.lower()])
  true = re.sub('[^0-9a-z ]', '', answer['true'.lower()])
  if pred == true:
    em.append(1)
  else:
    em.append(0)

In [48]:
# Accuracy
sum(em)/len(em)

1.0

## ROUGE

In [50]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from rouge import Rouge

In [52]:
model_out= 'hello to  the world'
ref = 'hello world'

In [53]:
rouge= Rouge()

In [56]:
rouge.get_scores(model_out, ref)

[{'rouge-1': {'f': 0.6666666622222223, 'p': 0.5, 'r': 1.0},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.6666666622222223, 'p': 0.5, 'r': 1.0}}]

In [57]:
answers

[{'predicted': 'Saint Bernadette Soubirous',
  'true': 'Saint Bernadette Soubirous'},
 {'predicted': 'a copper statue of Christ',
  'true': 'a copper statue of Christ'},
 {'predicted': 'the Main Building', 'true': 'the Main Building'},
 {'predicted': 'a Marian place of prayer and reflection.',
  'true': 'a Marian place of prayer and reflection'},
 {'predicted': 'a golden statue of the Virgin Mary.',
  'true': 'a golden statue of the Virgin Mary'}]

In [60]:
model_out = [ans['predicted'] for ans in answers]
ref = [ans['true'] for ans in answers]

model_out

['Saint Bernadette Soubirous',
 'a copper statue of Christ',
 'the Main Building',
 'a Marian place of prayer and reflection.',
 'a golden statue of the Virgin Mary.']

In [62]:
rouge.get_scores(model_out, ref, avg=True)

{'rouge-1': {'f': 0.9999999949999999, 'p': 1.0, 'r': 1.0},
 'rouge-2': {'f': 0.9999999949999999, 'p': 1.0, 'r': 1.0},
 'rouge-l': {'f': 0.9999999949999999, 'p': 1.0, 'r': 1.0}}

In [69]:
from tqdm import tqdm

model_out = []
ref = []

In [70]:
for pair in tqdm(squad[:10], leave=True):
  ans = qs({
      'question':pair['question'],
      'context':pair['context']
  })

  model_out.append(ans['answer'])
  ref.append(pair['answer'])

100%|██████████| 10/10 [00:22<00:00,  2.21s/it]


In [77]:
scores = rouge.get_scores(model_out, ref)

In [78]:
len(model_out), len(ref), len(scores)

(10, 10, 10)

In [80]:
print(model_out[4], ' | ', ref[4], ' | ', scores[4]['rouge-1']['f'])

a golden statue of the Virgin Mary.  |  a golden statue of the Virgin Mary  |  0.999999995


In [81]:
import re

clean = re.compile('(?i)[^0-9a-z ]')

model_out = [clean.sub('', text) for text in model_out]
ref = [clean.sub('', text) for text in ref]

In [82]:
scores = rouge.get_scores(model_out, ref, avg=True)

In [83]:
scores

{'rouge-1': {'f': 0.9499999951249999, 'p': 0.9333333333333332, 'r': 1.0},
 'rouge-2': {'f': 0.6999999964999999, 'p': 0.7, 'r': 0.7},
 'rouge-l': {'f': 0.9499999951249999, 'p': 0.9333333333333332, 'r': 1.0}}